# Figure 2

In [ ]:
using DifferentialEquations
using ModelingToolkit
using MitochondrialDynamics
using MitochondrialDynamics: GlcConst, G3P, Pyr, NADH_c, NADH_m, ATP_c, ADP_c, AMP_c, Ca_m, Ca_c, x, ΔΨm, degavg
import MitochondrialDynamics.Utils: second, μM, mV, mM, Hz
using Plots
pyplot()
Plots.pyrcparams["font.size"] = 14
Plots.pyrcparams["font.sans-serif"] = "Arial"
Plots.pyrcparams["font.family"] = "sans-serif"

In [ ]:
glc = range(3.0mM, 30.0mM, length=101)  # Range of glucose

@named sys = make_model()
idxGlc = findfirst(isequal(GlcConst), parameters(sys))
prob = SteadyStateProblem(sys, DEFAULT_U0)

In [ ]:
sols = map(glc) do g
    params = copy(prob.p)
    params[idxGlc] = g
    solve(remake(prob, p=params), DynamicSS(Rodas5()))
end

In [ ]:
extract(sols, k) = map(s->s[k], sols)

In [ ]:
glc5 = glc ./ 5
g3p = extract(sols, G3P) .* 1000
pyr = extract(sols, Pyr) .* 1000
ca_c = extract(sols, Ca_c) .* 1000
ca_m = extract(sols, Ca_m) .* 1000
nadh_c = extract(sols, NADH_c) .* 1000
nadh_m = extract(sols, NADH_m) .* 1000
atp_c = extract(sols, ATP_c) .* 1000
adp_c = extract(sols, ADP_c) .* 1000
amp_c = extract(sols, AMP_c) .* 1000
dpsi = extract(sols, ΔΨm) .* 1000
x1 = extract(sols, x[1])
x2 = extract(sols, x[2])
x3 = extract(sols, x[3])
deg = extract(sols, degavg)

In [ ]:
pl1 = plot(glc5, g3p, title="(A) G3P (μM)", ylim=(0.0, 8.0), lab=false)
pl2 = plot(glc5, pyr, title="(B) Pyruvate (μM)", ylim=(0.0, 75.0), lab=false)
pl3 = plot(glc5, [ca_c ca_m], label = ["cyto" "mito"], title="(C) Calcium (μM)", ylim=(0.0, 1.2), legend=:right)
pl4 = plot(glc5, [nadh_c nadh_m], label = ["cyto" "mito"], title="(D) NADH (μM)", legend=:right)
pl5 = plot(glc5, [atp_c adp_c amp_c], label = ["ATP" "ADP" "AMP"], title="(E) Adenylates (μM)", legend=:right)
pl6 = plot(glc5, atp_c ./ adp_c, title="(F) ATP/ADP ratio", legend=:right)
pl7 = plot(glc5, dpsi, title="(G) ΔΨ (mV)", label=false, ylim=(80, 150), xlabel="Glucose (X)")
pl8 = plot(glc5, [x1 x2 x3], label=["X1" "X2" "X3"], title="(H) Mitochondrial nodes", xlabel="Glucose (X)", legend=:right)
pl9 = plot(glc5, deg, label=false, title="(I) Average Node Degree", xlabel="Glucose (X)")
plot(pl1, pl2, pl3, pl4, pl5, pl6, pl7, pl8, pl9; size = (2000, 1500))

In [ ]:
# Uncomment if pdf file is required
# savefig("Fig2.pdf")